In [1]:
from pathlib import Path
import data_transformation as transformer
import crewai_mapper as mapper
import data_ingestion as ingestion

## 1. Data ingestion

In [3]:
INPUT_PATH = Path("./input_data/Openstaande posten crediteuren 300423 SAP.XLSX")

### Ingest validation rules

In [4]:
# These rules are easily configurable by a technical expert
VALIDATION_RULES = ingestion.VALIDATION_RULES

In [5]:
df = ingestion.file_ingestion(path_dir=INPUT_PATH)

Outputting input data to show original data read in from XLSX.

In [6]:
df

,Symbol Posten offen/ausg.,Zuordnung,Belegnummer,Belegart,Belegdatum,Sonderhauptb.Kennz.,Symbol Nettofälligkeit,Betrag in Hauswährung,Hauswährung,Ausgleichsbeleg,Text
0,NaN,6090466822,1.200402e+09,RE,2023-04-30,NaN,NaN,51671.15,EUR,NaN,NaN
1,NaN,6090466823,1.200402e+09,RE,2023-04-30,NaN,NaN,3919.22,EUR,NaN,NaN
2,@5C\Qoffen@,NaN,NaN,NaN,NaT,NaN,NaN,55590.37,EUR,NaN,NaN
3,Konto 30036349,NaN,NaN,NaN,NaT,NaN,NaN,55590.37,EUR,NaN,NaN
4,NaN,9503325616,1.200401e+09,RE,2023-04-25,NaN,NaN,-115.65,EUR,1.600014e+09,NaN
...,...,...,...,...,...,...,...,...,...,...,...
683,Konto 30139870,NaN,NaN,NaN,NaT,NaN,NaN,-165902.22,EUR,NaN,NaN
684,NaN,1283362112,1.100129e+09,KR,2023-04-12,NaN,NaN,-92.56,EUR,1.600014e+09,Jumbo-1283362112 keuken/kantoor
685,@5C\Qoffen@,NaN,NaN,NaN,NaT,NaN,NaN,-92.56,EUR,NaN,NaN
686,Konto 30140793,NaN,NaN,NaN,NaT,NaN,NaN,-92.56,EUR,NaN,NaN


## 2. Transform data

In [7]:
sub_df = transformer.transform_nonstandard_SAP(df=df, new_column_name="Cust. name")
csv = sub_df.head(10).to_csv(sep=',')

The `transformer` provides convenient functions to work on the data, such as adding new customer names.

In [8]:
sub_df

,Zuordnung,Belegnummer,Belegart,Belegdatum,Sonderhauptb.Kennz.,Symbol Nettofälligkeit,Betrag in Hauswährung,Hauswährung,Ausgleichsbeleg,Text,Cust. name
0,6090466822,1.200402e+09,RE,2023-04-30,NaN,NaN,51671.15,EUR,NaN,NaN,"(@5C\Qoffen@, Konto 30036349)"
1,6090466823,1.200402e+09,RE,2023-04-30,NaN,NaN,3919.22,EUR,NaN,NaN,"(@5C\Qoffen@, Konto 30036349)"
4,9503325616,1.200401e+09,RE,2023-04-25,NaN,NaN,-115.65,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30039682)"
5,9503328298,1.200401e+09,RE,2023-04-26,NaN,NaN,-216.41,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30039682)"
6,9503328299,1.200401e+09,RE,2023-04-26,NaN,NaN,-710.99,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30039682)"
...,...,...,...,...,...,...,...,...,...,...,...
678,0090861991,1.200402e+09,RE,2023-04-28,NaN,NaN,-1321.25,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30139870)"
679,0090861992,1.200402e+09,RE,2023-04-28,NaN,NaN,-104.66,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30139870)"
680,0090861993,1.200402e+09,RE,2023-04-28,NaN,NaN,-265.97,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30139870)"
681,0090861994,1.200402e+09,RE,2023-04-30,NaN,NaN,-3456.50,EUR,1.600014e+09,NaN,"(@5C\Qoffen@, Konto 30139870)"


### Retrieve Validation

In [9]:
DESTINATION_COLUMNS = VALIDATION_RULES["parametres"]["names"]
SOURCE_COLUMNS = sub_df.columns.to_list()
TO_DISCARD = VALIDATION_RULES["parametres"]["to_discard"]

## 3. CrewAI mapping suggestion

In [10]:
response = mapper.get_mapping(expected=DESTINATION_COLUMNS, input=csv)



> Entering new CrewAgentExecutor chain...
I need to work on the dataset to find the potential matches

Final Answer:
Datum --> Belegdatum
Bedrag_EUR --> Betrag in Hauswährung
Debiteurnaam --> Cust. name 
Factuurnummer --> Belegnummer

Note: The potential matches are the columns that have the same content as the input columns. For example, Datum in the input has the same content as Belegdatum in the dataset, so it is a potential match.

The content of the potential matches are:
- Cust. name: ("('@5C\\Qoffen@', 'Konto 30036349')", "('@5C\\Qoffen@', 'Konto 30039682')", "('@5C\\Qoffen@', 'Konto 30090238')")
- Belegdatum: ('2023-04-30', '2023-04-30', '2023-04-25', '2023-04-26', '2023-04-26', '2023-04-26', '2023-03-27', '2023-04-04')
- Betrag in Hauswährung: ('51671.15', '3919.22', '-115.65', '-216.41', '-710.99', '-227.38', '-3019.29', '-8079.25')
- Belegnummer: ('6090466822', '6090466823', '9503325616', '9503328298', '9503328299', '9503331521', '0096911384', '0096913057')

Therefore, the

## 4. Map data according to suggestion

In [11]:
source_columns = transformer.extract_substrings(response, SOURCE_COLUMNS)
destination_columns = transformer.extract_substrings(response, DESTINATION_COLUMNS)
mapping = transformer.create_mapping(source_columns=source_columns, 
                                     destination_columns=destination_columns)

In [14]:
mapping

{'Belegdatum': 'Datum',
 'Betrag in Hauswährung': 'Bedrag_EUR',
 'Cust. name': 'Debiteurnaam',
 'Belegnummer': 'Factuurnummer'}

In [16]:
mapped_df = transformer.mapper(mapping=mapping, to_discard=TO_DISCARD, df=sub_df)

After processing the suggestions fron the AI with the convenient built ins from the `transformer` we have a dataframe to use for mapping to a final output now.

In [17]:
mapped_df

,Factuurnummer,Datum,Bedrag_EUR,Debiteurnaam
0,1.200402e+09,2023-04-30,51671.15,"(@5C\Qoffen@, Konto 30036349)"
1,1.200402e+09,2023-04-30,3919.22,"(@5C\Qoffen@, Konto 30036349)"
4,1.200401e+09,2023-04-25,-115.65,"(@5C\Qoffen@, Konto 30039682)"
5,1.200401e+09,2023-04-26,-216.41,"(@5C\Qoffen@, Konto 30039682)"
6,1.200401e+09,2023-04-26,-710.99,"(@5C\Qoffen@, Konto 30039682)"
...,...,...,...,...
678,1.200402e+09,2023-04-28,-1321.25,"(@5C\Qoffen@, Konto 30139870)"
679,1.200402e+09,2023-04-28,-104.66,"(@5C\Qoffen@, Konto 30139870)"
680,1.200402e+09,2023-04-28,-265.97,"(@5C\Qoffen@, Konto 30139870)"
681,1.200402e+09,2023-04-30,-3456.50,"(@5C\Qoffen@, Konto 30139870)"
